In [ ]:
# !pip install bitsandbytes

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
import warnings
import os
from tqdm import tqdm

from transformers import BitsAndBytesConfig

warnings.simplefilter("ignore")

In [ ]:
path = '..'
os.chdir(path)
print(os.getcwd())

In [9]:
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "LLaMa-3-8b-LR1e4/"
model_id = "LLaMa-3-8b-LR5e5/"

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
def initiate_base_model(base_model_name):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        device_map="auto",
        torch_dtype="float16",
        quantization_config=bnb_config
    )

    model.config.use_cache = False

    tokenizer = AutoTokenizer.from_pretrained(base_model_name)

    tokenizer.pad_token_id = tokenizer.eos_token_id

    return model, tokenizer

In [10]:
model, tokenizer = initiate_base_model(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
messages = np.load('test_prompts.npy')

In [12]:
print(messages[0][1])


                Schema: CREATE TABLE "stadium" (
"Stadium_ID" int,
"Location" text,
"Name" text,
"Capacity" int,
"Highest" int,
"Lowest" int,
"Average" int,
PRIMARY KEY ("Stadium_ID")
)
CREATE TABLE "singer" (
"Singer_ID" int,
"Name" text,
"Country" text,
"Song_Name" text,
"Song_release_year" text,
"Age" int,
"Is_male" bool,
PRIMARY KEY ("Singer_ID")
)
CREATE TABLE "concert" (
"concert_ID" int,
"concert_Name" text,
"Theme" text,
"Stadium_ID" text,
"Year" text,
PRIMARY KEY ("concert_ID"),
FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID")
)
CREATE TABLE "singer_in_concert" (
"concert_ID" int,
"Singer_ID" text,
PRIMARY KEY ("concert_ID","Singer_ID"),
FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"),
FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID")
)
                Question: How many singers do we have?
                SQL Answer: 
                


In [13]:
responses = []
cont = 0

for message in tqdm(messages):
    prompt = [
        {
            "role": "system",
            "content": message[0]
        },
        {
            "role": "user",
            "content": message[1]
        }
    ]
    input_ids = tokenizer.apply_chat_template(prompt,
                                                add_generation_prompt=True,
                                                return_tensors="pt"
                                                ).to(model.device)
    terminators = [
                    tokenizer.eos_token_id,
                    tokenizer.convert_tokens_to_ids("<|eot_id|>")
                  ]
    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    responses.append(tokenizer.decode(response, skip_special_tokens=True))

    if(cont % 50 == 0):
        np.save('llama3-8b-LR5e5-answers.npy', responses)
np.save('llama3-8b-LR5e5-answers.npy', responses)

100%|██████████| 899/899 [1:32:09<00:00,  6.15s/it]
